In [1]:
#importing modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

from sklearn.datasets import load_iris

In [45]:
#load data
data = load_iris()
class_labels =list( data.target_names)
column_names = data.feature_names.extend(class_labels)

df = pd.DataFrame(columns=data.feature_names)
for i in range(len(data["data"])):
    features = list(data["data"][i]) #the main features
    label = data.target[[i]][0] #the class label, with 0, 1, or 2
    label_arr = [1 if label==0 else 0,1 if label==1 else 0,1 if label==2 else 0] #adding the class label on
    features.extend(label_arr)
    
    #inserting a new row into the df
    newSeries = pd.Series(features,index=df.columns)
    df = df.append(newSeries,ignore_index=True)
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),setosa,versicolor,virginica
0,5.1,3.5,1.4,0.2,1.0,0.0,0.0
1,4.9,3.0,1.4,0.2,1.0,0.0,0.0
2,4.7,3.2,1.3,0.2,1.0,0.0,0.0
3,4.6,3.1,1.5,0.2,1.0,0.0,0.0
4,5.0,3.6,1.4,0.2,1.0,0.0,0.0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0.0,0.0,1.0
146,6.3,2.5,5.0,1.9,0.0,0.0,1.0
147,6.5,3.0,5.2,2.0,0.0,0.0,1.0
148,6.2,3.4,5.4,2.3,0.0,0.0,1.0
